In [126]:
%%capture
from tqdm.auto import tqdm
from IPython.display import clear_output

import numpy as np
import pandas as pd

import torch
import biosppy
from scipy.signal import savgol_filter, resample

In [161]:
labels = pd.read_csv('task_1/train/train_gts.csv')
test_labels = pd.read_csv('task_1/test/test_meta.csv')[['record_name']]

In [96]:
new_labels = pd.DataFrame(columns=['record_name', 'myocard'])
filenames = labels.record_name

In [156]:
for filename in tqdm(filenames):
    data = np.load(f'../aiijc_data/task_1/train/{filename}.npy')
    data = np.apply_along_axis(lambda x: savgol_filter(x, 30, 2), axis=1, arr=data)

    peaks =  biosppy.signals.ecg.christov_segmenter(signal=data[10], sampling_rate = 500)[0]
    
    signals = []
    count = 1

    for i in peaks[1:-1]:
        x = i - 200
        y = i + 200

        signal = data[:, x:y]
        
        signals.append(signal)
        count += 1

    
    for i, signal in enumerate(signals):
        filepath = f'R_peaks/signals/{filename}_{i}'

        np.save(filepath, signal)
        
        res = pd.DataFrame(data={'record_name': f'{filename}_{i}', 'myocard': labels.loc[labels.record_name == filename, 'myocard']})
        new_labels = pd.concat([new_labels, res])

  0%|          | 0/2101 [00:00<?, ?it/s]

In [153]:
new_labels.to_csv('R_peaks/labels.csv', index=False)